In [ ]:
from dotenv import load_dotenv
from polygon import RESTClient
from datetime import datetime,timezone
import os

In [ ]:
import pandas as pd

In [ ]:
import numpy as np

In [ ]:

load_dotenv("../guidelight-api/.env")
token = os.getenv("POLYGON_TOKEN")

In [ ]:

client = RESTClient(api_key=token)

In [ ]:
ticker = "AAPL"

# List Aggregates (Bars)
aggs = []
for a in client.list_aggs(ticker=ticker, multiplier=1, timespan="minute", from_="2019-01-01", to="2023-12-30", limit=50000):
    aggs.append(a)

print(len(aggs))



In [ ]:
aggs[-1]

In [ ]:
#unix millisecond timestamp to datetime
# from pytz import timezone

# Convert timestamp to datetime in UTC timezone
def ts_to_dt(timestamp):
    return datetime.fromtimestamp(timestamp/1000, tz=timezone.utc)

timestamps = np.vectorize(ts_to_dt)

# Example usage
a = np.array([a.timestamp for a in aggs])
times = timestamps(a)
aapl_aggs = pd.DataFrame({"day": [time.strftime("%Y-%m-%d") for time in times],
                            "timestamp": [time.strftime("%Y-%m-%d, %H:%M") for time in times],
                           "close": [a.close for a in aggs], 
                           "volume": [a.volume for a in aggs],
                           "low": [a.low for a in aggs],
                           "high": [a.high for a in aggs]})



In [ ]:
from heston_param import *

In [ ]:
aapl_aggs.groupby("day")

In [ ]:
daily_aggs =  [aapl_aggs.groupby('day').get_group(x) for x in aapl_aggs['day'].unique()]

In [ ]:
hist_voilatilities = torch.empty(len(daily_aggs))
for i, day in enumerate(daily_aggs):
	hist_voilatilities[i] = estimate_historical_volatility(day['close'].values)

# calculate the historical volatility for each day





In [ ]:
import pickle

pickle.dump(hist_voilatilities, open("hist_voilatilities.pkl", "wb"))
pickle.dump(daily_aggs, open("daily_aggs.pkl", "wb"))

In [45]:
hist_voilatilities = pickle.load(open("hist_voilatilities.pkl", "rb"))
daily_aggs = pickle.load(open("daily_aggs.pkl", "rb"))

In [48]:
print(calibrate_daily_parameters(hist_voilatilities[0], 0.1, daily_aggs[0]["close"].values, 0.0237, 390, 3))

            day          timestamp    close   volume      low     high
0    2019-04-08  2019-04-08, 08:11  49.0675   2308.0  49.0675  49.0875
1    2019-04-08  2019-04-08, 08:13  49.0875    744.0  49.0875  49.0875
2    2019-04-08  2019-04-08, 08:26  49.0875    400.0  49.0875  49.0875
3    2019-04-08  2019-04-08, 08:34  49.0675   1200.0  49.0675  49.0750
4    2019-04-08  2019-04-08, 08:40  49.1225    688.0  49.1225  49.1225
..          ...                ...      ...      ...      ...      ...
609  2019-04-08  2019-04-08, 23:50  50.1125   8240.0  50.1125  50.1450
610  2019-04-08  2019-04-08, 23:55  50.1325   1444.0  50.1325  50.1350
611  2019-04-08  2019-04-08, 23:56  50.1500   8200.0  50.1325  50.1500
612  2019-04-08  2019-04-08, 23:57  50.1500  12600.0  50.1500  50.1500
613  2019-04-08  2019-04-08, 23:59  50.1450   4360.0  50.1425  50.1450

[614 rows x 6 columns]
